# Slackformer: Basic transformer on WBSlack data
All logic taken from Stacey's notebook and packaged into functions

In [1]:
import weave
import slack_trainer

# weave.use_frontend_devmode()

In [2]:
weave.get("local-artifact:///tmp/local-artifacts/typedDict/5a5c60639a6fd6711a0b555f068b2ccb")
# a = [1,2,3]
# a.index(2)

# get("local-artifact:///tmp/local-artifacts/typedDict/5a5c60639a6fd6711a0b555f068b2ccb")["usernames"][get("local-artifact:///tmp/local-artifacts/typedDict/5a5c60639a6fd6711a0b555f068b2ccb")["base_model"].call_string("tim tim tim tim 2").argmax]

# get("local-artifact:///tmp/local-artifacts/typedDict/5a5c60639a6fd6711a0b555f068b2ccb")["usernames"].kvToDict(get("local-artifact:///tmp/local-artifacts/typedDict/5a5c60639a6fd6711a0b555f068b2ccb")["base_model"].call_string("tim tim tim tim 2"))

2022-08-11 12:01:34.162100: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
SLACK_DATA_DIR = "/Users/timothysweeney/Documents/jan_1_2022_slack_dump"
raw_data = slack_trainer.load_data(SLACK_DATA_DIR)

In [3]:
config = {
    "min_msg_count"  : 100, 
    "train_frac"     : 0.8,
    "use_baseline"   : True,
    "vocab_size"     : 20000, 
    "sequence_length": 200,
    "embed_dim"      : 64,
    "num_heads"      : 6,
    "ff_dim"         : 64,
    "optimizer"      : "adam",
    "loss"           : "sparse_categorical_crossentropy",
    "metrics"        : ["accuracy"],
    "batch_size"     : 64, 
    "epochs"         : 1
}

In [4]:
# Process the data
data = slack_trainer.process_data(users         = raw_data["users"], 
                                  messages      = raw_data["messages"], 
                                  min_msg_count = config["min_msg_count"], 
                                  train_frac    = config["train_frac"])

In [5]:
# Build & Compile the Model
if config["use_baseline"]:
    model = slack_trainer.make_baseline_model(data, 
                                               vocab_size      = config["vocab_size"], 
                                               sequence_length = config["sequence_length"])
else:
    model = slack_trainer.make_transformer_model(data, 
                                                   vocab_size      = config["vocab_size"], 
                                                   sequence_length = config["sequence_length"],
                                                   embed_dim       = config["embed_dim"],
                                                   num_heads       = config["num_heads"],
                                                   ff_dim          = config["ff_dim"])
model.compile(optimizer = config["optimizer"], 
                       loss      = config["loss"], 
                       metrics   = config["metrics"])

2022-08-11 12:16:49.737637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Train the model
results = slack_trainer.fit_model(model, data, 
                                  batch_size = config["batch_size"], 
                                  epochs     = config["epochs"])

1452/1452 [==============================] - 4s 3ms/step - loss: 28.7352 - accuracy: 0.0375 - val_loss: 4.7419 - val_accuracy: 0.0443


In [7]:
# Package model for inference
packaged_model = slack_trainer.package_model(model, data)

In [8]:
# Save or Publish Model
saved_model = weave.save(packaged_model)

INFO:tensorflow:Assets written to: /tmp/local-artifacts/KerasModel/working-AQH6LTYFAFJS/_obj/assets


In [9]:
# Demonstrate calling
from weave.ecosystem import keras as weave_keras
weave_keras.call_string(saved_model, "i love weave")

1/1 [==============================] - 0s 125ms/step


In [10]:
weave_keras.call_string(weave.save(model), "i love weave")

1/1 [==============================] - 0s 76ms/step


In [10]:
weave.save({
    'model': model,
#     'packaged_model': packaged_model,
    'classes': data["users"].sort_values("model_id")["real_name"].tolist()
})

1/1 [==============================] - 0s 85ms/step


In [26]:
packaged_model.predict(["asdf"]).tolist()[0]

1/1 [==============================] - 0s 25ms/step


[b'Carey Phelps']

In [27]:
packaged_model.outputs

[<KerasTensor: shape=(None, 1) dtype=string (created by layer 'private_lookup_layer')>]